In [2]:
include("LiPoSID.jl")
using LinearAlgebra

Kraus operators for amplitude damping

$E_1 = \begin{bmatrix}
1 & 0 \\ 0 & \sqrt{1-\gamma}
\end{bmatrix}, \quad E_2 = \begin{bmatrix}
0 & \sqrt{\gamma} \\ 0 & 0 \end{bmatrix}$

In [3]:
γ = 0.1
E = []
push!(E, [1 0
          0 sqrt(1-γ)])
        
push!(E, [0 sqrt(γ)
          0  0 ])

sum(e'e for e in E)

2×2 Matrix{Float64}:
 1.0  0.0
 0.0  1.0

Set initial states:

$
\rho^{(g)}(0) =  \rho^{\ket{0}}(0) = \left(
\begin{array}{c}
            1  \\
            0   \\
        \end{array}
                \right) 
                \left(
                \begin{array} {cc}
                1 & 0
                \end{array}
                \right) = 
\ket{0} \bra{0} = \left(
        \begin{array}{ccccc}
            1  &  0  \\
            0  &  0   \\
        \end{array}
            \right), \notag \\
\rho^{(e)}(0)=\rho^{\ket{1}}(0)=\ket{1} \bra{1} = 
\begin{pmatrix}
0 \\ 1
\end{pmatrix}
\begin{pmatrix}
0 & 1
\end{pmatrix}=
\left(
        \begin{array}{ccccc}
            0  &  0  \\
            0  &  1  \\
        \end{array}
            \right), \notag\\
\rho^{\ket{x}}(0) =\ket{x} \bra{x} = |+\rangle \langle + | = \frac{1}{2}\left(
        \begin{array}{ccccc}
            1  &  1  \\
            1  &  1   \\
        \end{array}
    \right), \notag\\
\rho^{\ket{y}}(0)  = \ket{y} \bra{y} = |-\rangle \langle-| = \frac{1}{2}\left(
        \begin{array}{ccccc}
            1  &  -i  \\
            i  &  1   \\
        \end{array}
    \right)$

In [4]:
ρᵍ₀ = [1. 0]' * [1 0]
ρᵉ₀ = [0. 1]' * [0 1]
ρˣ₀ = [1  1]' * [1 1]/2
ρʸ₀ = [1 -im]' * [1 -im]/2

2×2 Matrix{ComplexF64}:
 0.5+0.0im  0.0-0.5im
 0.0+0.5im  0.5+0.0im

Kraus operator sum representation

$
\rho(t) = \sum_{k=1}^{s} E_k^{} \rho_0 E_k^{\dagger},
$

In [5]:
function Kraus(ρ₀, E)
    ρ = sum(K * ρ₀ * K' for K in E)
    return ρ
end

ρᵍ = Kraus(ρᵍ₀, E); @assert ishermitian(ρᵍ)
ρᵉ = Kraus(ρᵉ₀, E); @assert ishermitian(ρᵉ)
ρˣ = Kraus(ρˣ₀, E); @assert ishermitian(ρˣ)
ρʸ = Kraus(ρʸ₀, E); @assert ishermitian(ρʸ)

$
    \rho_1 = \begin{pmatrix}
        1 & 0\\
        0 & 0
    \end{pmatrix} = \ket{0}\bra{0}\\
    \rho_2 = \rho_1 X = \begin{pmatrix}
        1 & 0\\
        0 & 0
    \end{pmatrix} 
    \begin{pmatrix}
        0 & 1\\
        1 & 0
    \end{pmatrix} = \begin{pmatrix}
        0 & 1\\
        0 & 0
    \end{pmatrix} = \ket{0}\bra{1} \\
    \rho_3 = X \rho_1 =     \begin{pmatrix}
        0 & 1\\
        1 & 0
    \end{pmatrix}\begin{pmatrix}
        1 & 0\\
        0 & 0
    \end{pmatrix}  =
    \begin{pmatrix}
        0 & 0\\
        1 & 0
    \end{pmatrix} = \ket{1} \bra{0} \\
    \rho_4 = X \rho_1 X=     \begin{pmatrix}
        0 & 1\\
        1 & 0
    \end{pmatrix}\begin{pmatrix}
        1 & 0\\
        0 & 0
    \end{pmatrix} \begin{pmatrix}
        0 & 1\\
        1 & 0
    \end{pmatrix} = \begin{pmatrix}
        0 & 0\\
        1 & 0
    \end{pmatrix}\begin{pmatrix}
        0 & 1\\
        1 & 0
    \end{pmatrix} = 
    \begin{pmatrix}
        0 & 0\\
        0 & 1
    \end{pmatrix} = \ket{1} \bra{1}
$

Using Niesen-Chuang (8.154) formula:

$
\mathcal{E}(\ket{n}\bra{m}) =  \mathcal{E}(|+\rangle \langle + |) +i  \mathcal{E}(|-\rangle \langle-|) - \frac{1+i}{2}\mathcal{E}\ket{n}\bra{n} - \frac{1+i}{2}\mathcal{E}\ket{m}\bra{m},
$

$\rho_1' = \mathcal{E}(|0\rangle \langle 0 |)$

$\rho_4' = \mathcal{E}(|1\rangle \langle 1 |)$

$\rho_2' = \mathcal{E}(|0\rangle \langle 1 |) = \mathcal{E}(|+\rangle \langle + |) +i  \mathcal{E}(|-\rangle \langle-|) - (1+i) (\rho_1' + \rho_4')/2, $ 

$\rho_3'  = \mathcal{E}(|1\rangle \langle 0 |) = \mathcal{E}(|+\rangle \langle + |) -i  \mathcal{E}(|-\rangle \langle-|) - (1+i) (\rho_1' + \rho_4')/2$

Niesen-Chuang mix up vice-versa formulas (8.175) for $\rho_2'$ and (8.176) for $\rho_3'$ and also has "-" sign in expression $(1-i) (\rho_1' + \rho_4')$ in (8.175).

In [6]:
ρ₁ = ρᵍ
ρ₄ = ρᵉ
ρ₂ = ρˣ + im*ρʸ - (1+im)*(ρ₁+ρ₄)/2; # this matrix is not Hermitian
ρ₃ = ρˣ - im*ρʸ - (1-im)*(ρ₁+ρ₄)/2; # this matrix is not Hermitian

[ ρ₁ ρ₂ 
  ρ₃ ρ₄ ]

4×4 Matrix{ComplexF64}:
      1.0+0.0im  0.0+0.0im  0.0+0.0im  0.948683+0.0im
      0.0+0.0im  0.0+0.0im  0.0+0.0im       0.0+0.0im
      0.0+0.0im  0.0+0.0im  0.1+0.0im       0.0+0.0im
 0.948683+0.0im  0.0+0.0im  0.0+0.0im       0.9+0.0im

In [8]:
σₓ = [ 0  1
       1  0 ]  # X gate

Λ = [ I   σₓ
      σₓ  -I ] / 2 # was -I in Niesen-Chuang (8.178)

χ = Λ * [ ρ₁ ρ₂ 
          ρ₃ ρ₄ ] * Λ

#@assert ishermitian(χ) # it is not exactly Hermitian ???

4×4 Matrix{ComplexF64}:
 0.949342+0.0im     0.0+0.0im     0.0+0.0im        0.025+0.0im
      0.0+0.0im   0.025+0.0im  -0.025+0.0im          0.0+0.0im
      0.0+0.0im  -0.025+0.0im   0.025+0.0im          0.0+0.0im
    0.025+0.0im     0.0+0.0im     0.0+0.0im  0.000658351+0.0im

In [9]:
σ = [ [0 1; 1 0], [0 -im; im 0], [1 0; 0 -1], [1 0; 0 1] ]
σˣ = σ[1]; σʸ = σ[2]; σᶻ = σ[3]; σᴵ = σ[4]; 

E₀ = I(2)

E₁ = σˣ  #  σₓ  or X gate
E₂ = -im * σʸ
E₃ = σᶻ

Ẽ = [E₀, E₁, E₂, E₃] 

d, U = eigen(χ)
@assert U * diagm(d) * U' ≈ χ

Eᵗᵒᵐ = []
for i in 1:size(U)[2]
    Eᵢ = sqrt(Complex(d[i])) * sum(U[j,i] * Ẽ[j] for j in 1:size(U)[1])
    Eᵢ[abs.(Eᵢ) .< 1e-5] .= 0
    push!(Eᵗᵒᵐ, Eᵢ)
end

sum(e'e for e in Eᵗᵒᵐ)

2×2 Matrix{ComplexF64}:
 1.0+0.0im  0.0+0.0im
 0.0+0.0im  1.0+0.0im

In [41]:
Eᵗᵒᵐ 

4-element Vector{Any}:
 ComplexF64[0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im]
 ComplexF64[0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im]
 ComplexF64[0.0 + 0.0im 0.316227766016838 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im]
 ComplexF64[-1.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im -0.9486832980505137 + 0.0im]

In [10]:
E

2-element Vector{Any}:
 [1.0 0.0; 0.0 0.9486832980505138]
 [0.0 0.31622776601683794; 0.0 0.0]

In [11]:
sum(e'e for e in E)

2×2 Matrix{Float64}:
 1.0  0.0
 0.0  1.0

In [12]:
function choi(ρᵍ, ρᵉ, ρˣ, ρʸ)
    ρ₁ = ρᵍ
    ρ₄ = ρᵉ
    ρ₂ = ρˣ + im*ρʸ - (1+im)*(ρ₁+ρ₄)/2; # this matrix is not Hermitian
    ρ₃ = ρˣ - im*ρʸ - (1-im)*(ρ₁+ρ₄)/2; # this matrix is not Hermitian

    σₓ = [ 0  1
           1  0 ]  # X gate

    Λ = [ I   σₓ
        σₓ  -I ] / 2 # was -I in Niesen-Chuang (8.178)

    χ = Λ * [ ρ₁ ρ₂ 
              ρ₃ ρ₄ ] * Λ

    return χ
end

choi (generic function with 1 method)

In [14]:
function operator_sum(χ)

    #@assert ishermitian(χ)

    σ = [ [0 1; 1 0], [0 -im; im 0], [1 0; 0 -1], [1 0; 0 1] ]
    σˣ = σ[1]; σʸ = σ[2]; σᶻ = σ[3]; σᴵ = σ[4]; 

    E₀ = I(2)

    E₁ = σˣ  #  σₓ  or X gate
    E₂ = -im * σʸ
    E₃ = σᶻ

    Ẽ = [E₀, E₁, E₂, E₃] 

    d, U = eigen(χ)
    @assert U * diagm(d) * U' ≈ χ
    
    E = []
    for i in 1:size(U)[2]
        Eᵢ = sqrt(Complex(d[i])) * sum(U[j,i] * Ẽ[j] for j in 1:size(U)[1])
        Eᵢ[abs.(Eᵢ) .< 1e-5] .= 0
        push!(E, Eᵢ)
    end
    return E
end

Eᵗᵒᵐ = operator_sum(χ)

sum(e'e for e in Eᵗᵒᵐ)

2×2 Matrix{ComplexF64}:
 1.0+0.0im  0.0+0.0im
 0.0+0.0im  1.0+0.0im

In [15]:
ρᵍᵗᵒᵐ = Kraus(ρᵍ₀, Eᵗᵒᵐ); @assert ishermitian(ρᵍᵗᵒᵐ)
ρᵉᵗᵒᵐ = Kraus(ρᵉ₀, Eᵗᵒᵐ); @assert ishermitian(ρᵉᵗᵒᵐ)
ρˣᵗᵒᵐ = Kraus(ρˣ₀, Eᵗᵒᵐ); @assert ishermitian(ρˣᵗᵒᵐ)
ρʸᵗᵒᵐ = Kraus(ρʸ₀, Eᵗᵒᵐ); @assert ishermitian(ρʸᵗᵒᵐ)

In [16]:
function fidelity(ρ₁, ρ₂)
    abs((tr(sqrt(sqrt(ρ₁)*ρ₂*sqrt(ρ₁))))^2)
end

fidelity (generic function with 1 method)

In [59]:
@assert fidelity(ρᵍ, ρᵍᵗᵒᵐ) ≈ 1
@assert fidelity(ρᵉ, ρᵉᵗᵒᵐ) ≈ 1
@assert fidelity(ρˣ, ρˣᵗᵒᵐ) ≈ 1
@assert fidelity(ρʸ, ρʸᵗᵒᵐ) ≈ 1

Kraus operators for amplitude damping

$E_1 = \begin{bmatrix}
1 & 0 \\ 0 & \sqrt{1-\gamma}
\end{bmatrix}, \quad E_2 = \begin{bmatrix}
0 & \sqrt{\gamma} \\ 0 & 0 \end{bmatrix}$

In [18]:
for  γ in 0:.1:1

    print("γ = ", γ)
    print(" ")

    E = []
    push!(E, [1 0
              0 sqrt(1-γ)])
            
    push!(E, [0 sqrt(γ)
              0  0 ])
    
    ρᵍ = Kraus(ρᵍ₀, E); @assert ishermitian(ρᵍ)
    ρᵉ = Kraus(ρᵉ₀, E); @assert ishermitian(ρᵉ)
    ρˣ = Kraus(ρˣ₀, E); @assert ishermitian(ρˣ)
    ρʸ = Kraus(ρʸ₀, E); @assert ishermitian(ρʸ)

    χ = choi(ρᵍ, ρᵉ, ρˣ, ρʸ)
    print(ishermitian(χ))
    d, U = eigen(χ)
    print(d)

    Eᵗᵒᵐ = operator_sum(χ)

    ρᵍᵗᵒᵐ = Kraus(ρᵍ₀, Eᵗᵒᵐ); @assert ishermitian(ρᵍᵗᵒᵐ)
    ρᵉᵗᵒᵐ = Kraus(ρᵉ₀, Eᵗᵒᵐ); @assert ishermitian(ρᵉᵗᵒᵐ)
    ρˣᵗᵒᵐ = Kraus(ρˣ₀, Eᵗᵒᵐ); @assert ishermitian(ρˣᵗᵒᵐ)
    ρʸᵗᵒᵐ = Kraus(ρʸ₀, Eᵗᵒᵐ); @assert ishermitian(ρʸᵗᵒᵐ)

    fᵍ = fidelity(ρᵍ, ρᵍᵗᵒᵐ)
    fᵉ = fidelity(ρᵉ, ρᵉᵗᵒᵐ)
    fˣ = fidelity(ρˣ, ρˣᵗᵒᵐ)
    fʸ = fidelity(ρʸ, ρʸᵗᵒᵐ)

    println(min(fᵍ, fᵉ, fˣ, fʸ))

end


γ = 0.0 trueComplexF64[0.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, 1.0 + 0.0im]0.9999999999999993
γ = 0.1 true[4.163336342344337e-17, 7.771561172376096e-16, 0.05000000000000002, 0.95]0.9999999999999996
γ = 0.2 falseComplexF64[-6.015209364363391e-18 + 0.0im, 2.07746145073982e-33 + 0.0im, 0.09999999999999998 + 0.0im, 0.8999999999999999 + 0.0im]0.9999999999999996
γ = 0.3 falseComplexF64[0.0 + 0.0im, 1.591652623634883e-17 + 0.0im, 0.1499999999999999 + 0.0im, 0.8500000000000001 + 0.0im]0.9999999999999998
γ = 0.4 true[1.1102230246251565e-16, 7.771561172376096e-16, 0.2, 0.8]0.9999999999999996
γ = 0.5 falseComplexF64[0.0 + 0.0im, 1.9444170971836036e-17 + 0.0im, 0.24999999999999997 + 0.0im, 0.7500000000000002 + 0.0im]0.9999999999999996
γ = 0.6 falseComplexF64[-6.729301216572692e-19 + 0.0im, 0.0 + 0.0im, 0.2999999999999999 + 0.0im, 0.7000000000000005 + 0.0im]1.0
γ = 0.7 true[2.220446049250313e-16, 5.551115123125783e-16, 0.35000000000000014, 0.65]0.9999999999999991
γ = 0.8 true[2.220446049250313e-16, 

depolarizing chanel

$\sqrt{1-\frac{3p}{4}}\begin{bmatrix} 
1 & 0 \\ 0 & 1
\end{bmatrix}, \quad \sqrt{\frac{p}{4}}\begin{bmatrix}
0 & 1 \\ 1 & 0 \end{bmatrix},
 \quad \sqrt{\frac{p}{4}}\begin{bmatrix}
0 & -i \\ i & 0 \end{bmatrix},
 \quad \sqrt{\frac{p}{4}}\begin{bmatrix}
1 & 0 \\ 0 & -1 \end{bmatrix}$

In [71]:
for  p in 0:.1:1

    print("p = ", p)
    print(" ")

    E = []

    push!(E, sqrt(1-3p/4)σᴵ)
        
    push!(E, sqrt(p/4)σˣ)
    
    push!(E, sqrt(p/4)σʸ)
         
    push!(E, sqrt(p/4)σᶻ)
    
    ρᵍ = Kraus(ρᵍ₀, E); @assert ishermitian(ρᵍ)
    ρᵉ = Kraus(ρᵉ₀, E); @assert ishermitian(ρᵉ)
    ρˣ = Kraus(ρˣ₀, E); @assert ishermitian(ρˣ)
    ρʸ = Kraus(ρʸ₀, E); @assert ishermitian(ρʸ)

    χ = choi(ρᵍ, ρᵉ, ρˣ, ρʸ)

    Eᵗᵒᵐ = operator_sum(χ)

    ρᵍᵗᵒᵐ = Kraus(ρᵍ₀, Eᵗᵒᵐ); #@assert ishermitian(ρᵍᵗᵒᵐ)
    ρᵉᵗᵒᵐ = Kraus(ρᵉ₀, Eᵗᵒᵐ); #@assert ishermitian(ρᵉᵗᵒᵐ)
    ρˣᵗᵒᵐ = Kraus(ρˣ₀, Eᵗᵒᵐ); #@assert ishermitian(ρˣᵗᵒᵐ)
    ρʸᵗᵒᵐ = Kraus(ρʸ₀, Eᵗᵒᵐ); #@assert ishermitian(ρʸᵗᵒᵐ)

    fᵍ = fidelity(ρᵍ, ρᵍᵗᵒᵐ)
    fᵉ = fidelity(ρᵉ, ρᵉᵗᵒᵐ)
    fˣ = fidelity(ρˣ, ρˣᵗᵒᵐ)
    fʸ = fidelity(ρʸ, ρʸᵗᵒᵐ)

    println(min(fᵍ, fᵉ, fˣ, fʸ))

end

p = 0.0 0.9999999999999993
p = 0.1 0.9999999999999993
p = 0.2 1.0
p = 0.3 0.9999999999999987
p = 0.4 0.9999999999999989
p = 0.5 0.9999999999999996
p = 0.6 0.9999999999999993
p = 0.7 1.0
p = 0.8 0.9999999999999987
p = 0.9 0.9999999999999989
p = 1.0 1.0


Phase damping

$E_1 = \begin{bmatrix}
1 & 0 \\ 0 & \sqrt{1-\gamma}
\end{bmatrix}, \quad E_2 = \begin{bmatrix}
0 &  0 \\ 0 & \sqrt{\gamma} \end{bmatrix}$

In [72]:
for  γ in 0:.1:1

    print("γ = ", γ)
    print(" ")

    E = []
    push!(E, [1 0
              0 sqrt(1-γ)])
            
    push!(E, [0  0
              0  sqrt(γ) ])
    
    ρᵍ = Kraus(ρᵍ₀, E); @assert ishermitian(ρᵍ)
    ρᵉ = Kraus(ρᵉ₀, E); @assert ishermitian(ρᵉ)
    ρˣ = Kraus(ρˣ₀, E); @assert ishermitian(ρˣ)
    ρʸ = Kraus(ρʸ₀, E); @assert ishermitian(ρʸ)

    χ = choi(ρᵍ, ρᵉ, ρˣ, ρʸ)

    Eᵗᵒᵐ = operator_sum(χ)

    ρᵍᵗᵒᵐ = Kraus(ρᵍ₀, Eᵗᵒᵐ); @assert ishermitian(ρᵍᵗᵒᵐ)
    ρᵉᵗᵒᵐ = Kraus(ρᵉ₀, Eᵗᵒᵐ); @assert ishermitian(ρᵉᵗᵒᵐ)
    ρˣᵗᵒᵐ = Kraus(ρˣ₀, Eᵗᵒᵐ); @assert ishermitian(ρˣᵗᵒᵐ)
    ρʸᵗᵒᵐ = Kraus(ρʸ₀, Eᵗᵒᵐ); @assert ishermitian(ρʸᵗᵒᵐ)

    fᵍ = fidelity(ρᵍ, ρᵍᵗᵒᵐ)
    fᵉ = fidelity(ρᵉ, ρᵉᵗᵒᵐ)
    fˣ = fidelity(ρˣ, ρˣᵗᵒᵐ)
    fʸ = fidelity(ρʸ, ρʸᵗᵒᵐ)

    println(min(fᵍ, fᵉ, fˣ, fʸ))

end

γ = 0.0 0.9999999999999993
γ = 0.1 0.9999999999999991
γ = 0.2 0.9999999999999973
γ = 0.3 1.0
γ = 0.4 0.9999999999999991
γ = 0.5 0.9999999999999989
γ = 0.6 0.9999999999999993
γ = 0.7 0.9999999999999996
γ = 0.8 0.9999999999999996
γ = 0.9 0.9999999999999998
γ = 1.0 1.0


Phase flip

$E_1 = \sqrt{p}\begin{bmatrix}
1 & 0 \\ 0 & 1
\end{bmatrix}, \quad E_2 = \sqrt{1-p}\begin{bmatrix}
1 &  0 \\ 0 & -1 \end{bmatrix}$

In [73]:
for  p in 0:.1:1

    print("p = ", p)
    print(" ")

    E = []

    push!(E, sqrt(p)σᴵ)
               
    push!(E, sqrt(1-p)σᶻ)
    
    ρᵍ = Kraus(ρᵍ₀, E); @assert ishermitian(ρᵍ)
    ρᵉ = Kraus(ρᵉ₀, E); @assert ishermitian(ρᵉ)
    ρˣ = Kraus(ρˣ₀, E); @assert ishermitian(ρˣ)
    ρʸ = Kraus(ρʸ₀, E); @assert ishermitian(ρʸ)

    χ = choi(ρᵍ, ρᵉ, ρˣ, ρʸ)

    Eᵗᵒᵐ = operator_sum(χ)

    ρᵍᵗᵒᵐ = Kraus(ρᵍ₀, Eᵗᵒᵐ); #@assert ishermitian(ρᵍᵗᵒᵐ)
    ρᵉᵗᵒᵐ = Kraus(ρᵉ₀, Eᵗᵒᵐ); #@assert ishermitian(ρᵉᵗᵒᵐ)
    ρˣᵗᵒᵐ = Kraus(ρˣ₀, Eᵗᵒᵐ); #@assert ishermitian(ρˣᵗᵒᵐ)
    ρʸᵗᵒᵐ = Kraus(ρʸ₀, Eᵗᵒᵐ); #@assert ishermitian(ρʸᵗᵒᵐ)

    fᵍ = fidelity(ρᵍ, ρᵍᵗᵒᵐ)
    fᵉ = fidelity(ρᵉ, ρᵉᵗᵒᵐ)
    fˣ = fidelity(ρˣ, ρˣᵗᵒᵐ)
    fʸ = fidelity(ρʸ, ρʸᵗᵒᵐ)

    println(min(fᵍ, fᵉ, fˣ, fʸ))

end

p = 0.0 0.9999999999999993
p = 0.1 0.9999999999999996
p = 0.2 0.9999999999999987
p = 0.3 0.9999999999999996
p = 0.4 0.9999999999999991
p = 0.5 1.0
p = 0.6 0.9999999999999991
p = 0.7 0.9999999999999996
p = 0.8 0.9999999999999987
p = 0.9 0.9999999999999996
p = 1.0 0.9999999999999993


bit flip

$E_1 = \sqrt{p}\begin{bmatrix}
1 & 0 \\ 0 & 1
\end{bmatrix}, \quad E_2 = \sqrt{1-p}\begin{bmatrix}
0 &  1 \\ 1 & 0 \end{bmatrix}$

In [74]:
for  p in 0:.1:1

    print("p = ", p)
    print(" ")

    E = []

    push!(E, sqrt(p)σᴵ)

    push!(E, sqrt(1-p)σˣ)
    
    ρᵍ = Kraus(ρᵍ₀, E); @assert ishermitian(ρᵍ)
    ρᵉ = Kraus(ρᵉ₀, E); @assert ishermitian(ρᵉ)
    ρˣ = Kraus(ρˣ₀, E); @assert ishermitian(ρˣ)
    ρʸ = Kraus(ρʸ₀, E); @assert ishermitian(ρʸ)

    χ = choi(ρᵍ, ρᵉ, ρˣ, ρʸ)

    Eᵗᵒᵐ = operator_sum(χ)

    ρᵍᵗᵒᵐ = Kraus(ρᵍ₀, Eᵗᵒᵐ); #@assert ishermitian(ρᵍᵗᵒᵐ)
    ρᵉᵗᵒᵐ = Kraus(ρᵉ₀, Eᵗᵒᵐ); #@assert ishermitian(ρᵉᵗᵒᵐ)
    ρˣᵗᵒᵐ = Kraus(ρˣ₀, Eᵗᵒᵐ); #@assert ishermitian(ρˣᵗᵒᵐ)
    ρʸᵗᵒᵐ = Kraus(ρʸ₀, Eᵗᵒᵐ); #@assert ishermitian(ρʸᵗᵒᵐ)

    fᵍ = fidelity(ρᵍ, ρᵍᵗᵒᵐ)
    fᵉ = fidelity(ρᵉ, ρᵉᵗᵒᵐ)
    fˣ = fidelity(ρˣ, ρˣᵗᵒᵐ)
    fʸ = fidelity(ρʸ, ρʸᵗᵒᵐ)

    println(min(fᵍ, fᵉ, fˣ, fʸ))

end

p = 0.0 0.9999999999999993
p = 0.1 0.9999999999999987
p = 0.2 0.9999999999999987
p = 0.3 0.9999999999999993
p = 0.4 0.9999999999999991
p = 0.5 0.9999999999999993
p = 0.6 0.9999999999999991
p = 0.7 0.9999999999999993
p = 0.8 0.9999999999999987
p = 0.9 0.9999999999999987
p = 1.0 0.9999999999999993


bit flip

$E_1 = \sqrt{p}\begin{bmatrix}
1 & 0 \\ 0 & 1
\end{bmatrix}, \quad E_2 = \sqrt{1-p}\begin{bmatrix}
0 &  -i \\ i & 0 \end{bmatrix}$

In [75]:
for  p in 0:.1:1

    print("p = ", p)
    print(" ")

    E = []

    push!(E, sqrt(p)σᴵ)

    push!(E, sqrt(1-p)σʸ)
    
    ρᵍ = Kraus(ρᵍ₀, E); @assert ishermitian(ρᵍ)
    ρᵉ = Kraus(ρᵉ₀, E); @assert ishermitian(ρᵉ)
    ρˣ = Kraus(ρˣ₀, E); @assert ishermitian(ρˣ)
    ρʸ = Kraus(ρʸ₀, E); @assert ishermitian(ρʸ)

    χ = choi(ρᵍ, ρᵉ, ρˣ, ρʸ)

    Eᵗᵒᵐ = operator_sum(χ)

    ρᵍᵗᵒᵐ = Kraus(ρᵍ₀, Eᵗᵒᵐ); #@assert ishermitian(ρᵍᵗᵒᵐ)
    ρᵉᵗᵒᵐ = Kraus(ρᵉ₀, Eᵗᵒᵐ); #@assert ishermitian(ρᵉᵗᵒᵐ)
    ρˣᵗᵒᵐ = Kraus(ρˣ₀, Eᵗᵒᵐ); #@assert ishermitian(ρˣᵗᵒᵐ)
    ρʸᵗᵒᵐ = Kraus(ρʸ₀, Eᵗᵒᵐ); #@assert ishermitian(ρʸᵗᵒᵐ)

    fᵍ = fidelity(ρᵍ, ρᵍᵗᵒᵐ)
    fᵉ = fidelity(ρᵉ, ρᵉᵗᵒᵐ)
    fˣ = fidelity(ρˣ, ρˣᵗᵒᵐ)
    fʸ = fidelity(ρʸ, ρʸᵗᵒᵐ)

    println(min(fᵍ, fᵉ, fˣ, fʸ))

end

p = 0.0 0.9999999999999993
p = 0.1 0.9999999999999987
p = 0.2 0.9999999999999987
p = 0.3 0.9999999999999993
p = 0.4 0.9999999999999991
p = 0.5 0.9999999999999993
p = 0.6 0.9999999999999991
p = 0.7 0.9999999999999993
p = 0.8 0.9999999999999987
p = 0.9 0.9999999999999987
p = 1.0 0.9999999999999993
